# Install AWS CLI

In [3]:
#! brew upgrade awscli
! aws --version

aws-cli/2.22.12 Python/3.12.7 Darwin/23.5.0 source/arm64


# Spark

In [1]:
from pyspark.sql import SparkSession
import os

# Set JAVA_HOME if needed
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"

# Define variables
WAREHOUSE_PATH = "XXXX"

# Initialize SparkSession with required configurations
spark = SparkSession.builder \
    .appName("iceberg_lab") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0,software.amazon.s3tables:s3-tables-catalog-for-iceberg-runtime:0.1.3,software.amazon.awssdk:glue:2.20.143,software.amazon.awssdk:sts:2.20.143,software.amazon.awssdk:s3:2.20.143,software.amazon.awssdk:dynamodb:2.20.143,software.amazon.awssdk:kms:2.20.143")  \
    .config("spark.sql.catalog.s3tablesbucket", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.s3tablesbucket.client.region", "us-east-1") \
    .config("spark.sql.catalog.defaultCatalog", "s3tablesbucket") \
    .config("spark.sql.catalog.s3tablesbucket.warehouse", WAREHOUSE_PATH) \
    .config("spark.sql.catalog.s3tablesbucket.catalog-impl","software.amazon.s3tables.iceberg.S3TablesCatalog") \
    .getOrCreate()


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/soumilshah/.ivy2/cache
The jars for the packages stored in: /Users/soumilshah/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
software.amazon.s3tables#s3-tables-catalog-for-iceberg-runtime added as a dependency
software.amazon.awssdk#glue added as a dependency
software.amazon.awssdk#sts added as a dependency
software.amazon.awssdk#s3 added as a dependency
software.amazon.awssdk#dynamodb added as a dependency
software.amazon.awssdk#kms added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13f0a66f-6465-4a0c-927d-665f29c4f533;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 in central
	found software.amazon.s3tables#s3-tables-catalog-for-iceberg-runtime;0.1.3 in central
	found software.amazon.awssdk#glue;2.20.143 in central
	found software.amazon.awssdk#aws-json-protocol;2.20.143 in central
	found software.amazon.awssdk#aws-core;2.20.143 in central
	fo

In [3]:
spark

In [2]:
spark.sql("SHOW NAMESPACES IN s3tablesbucket").show()
spark.sql("USE s3tablesbucket.example_namespace").show()
spark.sql("SHOW TABLES").show()

+-----------------+
|        namespace|
+-----------------+
|example_namespace|
+-----------------+

++
||
++
++

+-----------------+-------------+-----------+
|        namespace|    tableName|isTemporary|
+-----------------+-------------+-----------+
|example_namespace|bronze_orders|      false|
|example_namespace|silver_orders|      false|
+-----------------+-------------+-----------+


In [3]:
spark.sql("SELECT count(*) FROM bronze_orders ").show()
spark.sql("SELECT count(*) FROM silver_orders ").show()

+--------+
|count(1)|
+--------+
|       7|
+--------+

+--------+
|count(1)|
+--------+
|       5|
+--------+


In [4]:

spark.sql("SELECT invoiceid,category FROM bronze_orders ").show()
spark.sql("SELECT invoiceid,category FROM silver_orders ").show()


+---------+------------+
|invoiceid|    category|
+---------+------------+
|    40994|  market####|
|    83597|language####|
|    24137|      degree|
|    15587|         bit|
|    42918|      school|
|    40994|      market|
|    83597|    language|
+---------+------------+


+---------+------------+
|invoiceid|    category|
+---------+------------+
|    42918|      school|
|    15587|         bit|
|    24137|      degree|
|    83597|language####|
|    40994|  market####|
+---------+------------+
